Import modules.


In [1]:
import os
import time

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf

import chiron


Disable visibility of all GPU devices.


In [2]:
chiron.set_visible_gpus()


Load dataset from TFRecord file.


In [3]:
dataset = chiron.load_tfrecord(
    "../data/brain-tumor-public-dataset/tfrecord/train/fold-1.tfrecord"
)
dataset.element_spec


(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.string, name=None))

Define mapping function parameters.


In [4]:
shuffle_buffer_size = 512
seed = 0
label_map = {"meningioma": 0, "glioma": 1, "pituitary": 2}
num_classes = 3
image_size = [224, 224]
num_channels = 3
batch_size = 32


Map labels to class indices.


In [5]:
label_mapper = chiron.LabelMapper(label_map)


One-hot encode class indices.


In [6]:
one_hot_encoder = chiron.OneHotEncoder(num_classes)


Resize images so that they may be batched.


In [7]:
resizer = chiron.Resizer(image_size)


Stack images along channels axis.


In [8]:
repeater = chiron.Repeater(num_channels, axis=2)


Set image shape.


In [9]:
shape_setter = chiron.ShapeSetter(image_size + [num_channels])


Standardize images across batch.


In [10]:
whitener = chiron.PerBatchStandardWhitener()


Define naive data preprocessing pipeline.


In [11]:
dataset_naive = (
    dataset.shuffle(shuffle_buffer_size, seed=seed)
    .map(label_mapper)
    .map(one_hot_encoder)
    .map(resizer)
    .map(repeater)
    .map(shape_setter)
    .batch(batch_size)
    .map(whitener)
)


In [12]:
dataset_naive.take(1)


<TakeDataset shapes: ((None, 224, 224, 3), (None, 3)), types: (tf.float32, tf.float32)>

Define optimized data preprocessing pipeline. Parallelizes data transformations, caches dataset in memory, and prefetches elements.


In [13]:
dataset_optimized = (
    dataset.shuffle(shuffle_buffer_size, seed=seed)
    .map(label_mapper, num_parallel_calls=tf.data.AUTOTUNE)
    .map(one_hot_encoder, num_parallel_calls=tf.data.AUTOTUNE)
    .map(resizer, num_parallel_calls=tf.data.AUTOTUNE)
    .map(repeater, num_parallel_calls=tf.data.AUTOTUNE)
    .map(shape_setter, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .map(whitener, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)


Benchmark performance.


In [14]:
# https://www.tensorflow.org/guide/data_performance#the_training_loop

def benchmark(dataset):
    start_time = time.perf_counter()
    for _ in range(5):
        for _ in dataset:
            time.sleep(0.01)  # Simulate training time
    return time.perf_counter() - start_time


result_naive = benchmark(dataset_naive)
result_optimized = benchmark(dataset_optimized)
speedup = result_naive / result_optimized

print(f"Naive: {result_naive:.2f}s")
print(f"Optimized: {result_optimized:.2f}s")
print(f"Speedup: x{speedup:.2f}")


Naive: 27.91s
Optimized: 5.94s
Speedup: x4.70
